# RM-Gallery Bench 用户使用教程

本教程将介绍如何使用 RM-Gallery 的 EvaluationRunner 进行模型输出的批量评测。我们将涵盖以下内容：
1. EvaluationRunner 的基本概念
2. 如何配置和使用 EvaluationRunner
3. 如何准备评测数据
4. 如何运行评测并分析结果

## 什么是 EvaluationRunner？

EvaluationRunner 是 RM-Gallery 中用于批量运行评估任务的组件。它允许用户并行地对多个数据样本进行评估，支持多种评估器（Grader）同时运行，并提供了并发控制功能。

让我们首先导入必要的模块：

In [ ]:
import asyncio
from typing import List

from rm_gallery.core.data import DataSample, validate_data_samples
from rm_gallery.core.grader import GraderMode, LLMGrader, FunctionGrader, GraderScore
from rm_gallery.core.model.template import Template, RequiredField
from rm_gallery.core.runner.evaluation import EvaluationRunner
from dotenv import dotenv_values
dotenv_values()
print("Loading environment variables...")

Loading environment variables...


## 1. 创建评估器（Graders）

在使用 EvaluationRunner 之前，我们需要定义一个或多个评估器。评估器是实际执行评估逻辑的组件。

### 1.1 创建基于函数的评估器

In [ ]:
# 使用装饰器创建一个简单的基于函数的评估器
@FunctionGrader.wrap
async def length_grader(answer: str, **kwargs) -> GraderScore:
    """根据回答长度评分的评估器 - 更长的回答得分更高"""
    # 简单地根据字符数评分，归一化到0-1之间
    score = min(len(answer) / 100.0, 1.0)  # 假设100个字符为满分
    return GraderScore(
        score=score,
        reason=f"回答长度为 {len(answer)} 个字符"
    )

# 创建另一个基于函数的评估器
@FunctionGrader.wrap
async def keyword_grader(answer: str, query: str, **kwargs) -> GraderScore:
    """检查回答中是否包含查询关键词的评估器"""
    # 提取查询中的关键词（简化处理）
    keywords = query.lower().split()
    answer_lower = answer.lower()
    
    # 计算包含的关键词比例
    matched_keywords = sum(1 for keyword in keywords if keyword in answer_lower)
    score = matched_keywords / len(keywords) if keywords else 1.0
    
    return GraderScore(
        score=score,
        reason=f"查询中的 {len(keywords)} 个关键词中有 {matched_keywords} 个在回答中找到"
    )

print("基于函数的评估器创建完成！")

基于函数的评估器创建完成！


### 1.2 创建基于LLM的评估器

对于更复杂的评估任务，我们可以使用基于LLM的评估器：

In [ ]:
# 定义一个基于LLM的评估器模板
quality_template = {
    "messages": [
        {
            "role": "system",
            "content": (
                "你是一个专业的评估助手，负责评估AI模型回答的质量。请根据以下标准评估回答：\n"
                "1. 相关性：回答是否与问题相关\n"
                "2. 准确性：回答是否准确无误\n"
                "3. 完整性：回答是否完整地解答了问题\n"
                "请给出0到1之间的分数，1表示完美回答，0表示完全不相关或错误的回答。"
            )
        },
        {
            "role": "user",
            "content": (
                "问题：{query}\n"
                "回答：{answer}\n\n"
                "请根据上述标准评估这个回答的质量，并以以下JSON格式输出：\n"
                "{{\n  \"score\": <0到1之间的分数>,\n  \"reason\": \"评估理由\"\n}}"
            )
        }
    ],
    "required_fields": [
        RequiredField(
            name="query",
            type="string",
            position="data",
            description="用户提出的问题"
        ),
        RequiredField(
            name="answer",
            type="string",
            position="sample",
            description="模型的回答"
        )
    ]
}

# 定义模型配置（这里使用占位符配置）
model_config = {
    "model_name": "qwen-plus",
    "stream": False,
    "client_args": {
        "timeout": 60,
    },
}

# 创建基于LLM的评估器
quality_grader = LLMGrader(
    name="quality_grader",
    mode=GraderMode.POINTWISE,
    description="基于LLM的回答质量评估器",
    template=Template(**quality_template),
    model=model_config,
    rubrics=""
)

print("基于LLM的评估器创建完成！")

基于LLM的评估器创建完成！


## 2. 准备评测数据

现在我们需要准备一些测试数据。在 RM-Gallery 中，数据以 DataSample 对象的形式组织：

In [ ]:
# 定义数据样本的 schema
data_sample_schema = {
    "type": "object",
    "properties": {
        "data": {
            "type": "object",
            "properties": {
                "query": {"type": "string"},
            },
            "required": ["query"],
        },
        "samples": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {"answer": {"type": "string"}},
                "required": ["answer"],
            },
        },
    },
    "required": ["data", "samples"],
}

# 创建测试数据
raw_data_samples = [
    {
        "data": {
            "query": "什么是人工智能？",
        },
        "samples": [
            {"answer": "人工智能是计算机科学的一个分支，它企图了解智能的实质，并生产出一种新的能以人类智能相似的方式做出反应的智能机器。"},
            {"answer": "AI是计算机科学的一个领域。"},
            {"answer": "人工智能就是让机器像人一样思考和学习的技术。它包括机器学习、深度学习、自然语言处理等技术。"}
        ],
    },
    {
        "data": {
            "query": "Python语言的特点有哪些？",
        },
        "samples": [
            {"answer": "Python是一种解释型、面向对象、动态数据类型的高级程序设计语言。Python具有简洁、易读的语法，使得代码更容易维护和理解。"},
            {"answer": "Python简单易学，有丰富的库支持。"},
            {"answer": ""}  # 空回答，用于测试边界情况
        ],
    }
]

# 验证并转换为 DataSample 对象
data_samples = validate_data_samples(raw_data_samples, data_sample_schema)

print(f"准备了 {len(data_samples)} 个测试样本")
print(f"第一个样本包含 {len(data_samples[0].samples)} 个回答")
print(f"第二个样本包含 {len(data_samples[1].samples)} 个回答")

准备了 2 个测试样本
第一个样本包含 3 个回答
第二个样本包含 3 个回答


## 3. 配置和使用 EvaluationRunner

现在我们已经准备好了评估器和测试数据，可以配置 EvaluationRunner 来运行评估任务了。

### 3.1 配置评估器

In [ ]:
# 配置评估器
grader_configs = {
    "length_evaluator": {
        "grader": length_grader(),
    },
    "keyword_evaluator": {
        "grader": keyword_grader(),
    },
    "quality_evaluator": {
        "grader": quality_grader,
    }
}

print("评估器配置完成！")

评估器配置完成！


### 3.2 创建和运行 EvaluationRunner

In [ ]:
# 创建 EvaluationRunner 实例
# 设置最大并发数为2，避免过多的并发请求
runner = EvaluationRunner(grader_configs=grader_configs, max_concurrent=2)

print("EvaluationRunner 创建完成！")
print(f"配置了 {len(runner.grader_configs)} 个评估器")

EvaluationRunner 创建完成！
配置了 3 个评估器


### 3.3 运行评估任务

In [ ]:
# 运行评估任务
# 注意：由于我们使用了基于LLM的评估器，如果要实际运行此代码，需要配置有效的API密钥
# 在这个示例中，我们只展示代码结构

print("开始运行评估任务...")
print("注意：如果要实际运行基于LLM的评估器，需要配置有效的API密钥")

# 如果你有有效的API密钥，可以取消注释下面的代码来实际运行评估
results = await runner(data_samples=data_samples)
print("评估完成！")
print(f"结果: {results}")

print("\n评估任务结构演示完成！")

开始运行评估任务...
注意：如果要实际运行基于LLM的评估器，需要配置有效的API密钥


2025-11-11 20:41:48.319 | INFO     | rm_gallery.core.runner.evaluation:__call__:90 - Results: [{'length_evaluator': [GraderScore(reason='回答长度为 56 个字符', metadata={}, score=0.56), GraderScore(reason='回答长度为 14 个字符', metadata={}, score=0.14), GraderScore(reason='回答长度为 45 个字符', metadata={}, score=0.45)], 'keyword_evaluator': [GraderScore(reason='查询中的 1 个关键词中有 0 个在回答中找到', metadata={}, score=0.0), GraderScore(reason='查询中的 1 个关键词中有 0 个在回答中找到', metadata={}, score=0.0), GraderScore(reason='查询中的 1 个关键词中有 0 个在回答中找到', metadata={}, score=0.0)], 'quality_evaluator': [GraderScore(reason='回答准确地定义了人工智能，指出它是计算机科学的一个分支，目标是理解智能本质并制造能模拟人类智能行为的机器。内容相关、表述清晰且信息完整，但可以进一步补充一些典型应用或技术（如机器学习、自然语言处理）以提升完整性。', metadata={}, score=0.9), GraderScore(reason='回答部分相关，指出人工智能（AI）是计算机科学的一个领域，这是正确的。但回答过于简略，缺乏对人工智能具体含义、目标或应用的解释，未能完整解答‘什么是人工智能’这一问题。一个更完整的回答应包括让机器模拟人类智能行为（如学习、推理、识别等）的内容。因此，相关性尚可，准确性基本成立，但完整性不足。', metadata={}, score=0.4), GraderScore(reason='回答准确且相关，简明扼要地解释了人工智能的核心概念——让机器像人一样思考和学习，并列举了关键技术如机器学习、深度学习和自然语言处理，体现了完整性。

评估完成！
结果: {'results': [{'length_evaluator': [GraderScore(reason='回答长度为 56 个字符', metadata={}, score=0.56), GraderScore(reason='回答长度为 14 个字符', metadata={}, score=0.14), GraderScore(reason='回答长度为 45 个字符', metadata={}, score=0.45)], 'keyword_evaluator': [GraderScore(reason='查询中的 1 个关键词中有 0 个在回答中找到', metadata={}, score=0.0), GraderScore(reason='查询中的 1 个关键词中有 0 个在回答中找到', metadata={}, score=0.0), GraderScore(reason='查询中的 1 个关键词中有 0 个在回答中找到', metadata={}, score=0.0)], 'quality_evaluator': [GraderScore(reason='回答准确地定义了人工智能，指出它是计算机科学的一个分支，目标是理解智能本质并制造能模拟人类智能行为的机器。内容相关、表述清晰且信息完整，但可以进一步补充一些典型应用或技术（如机器学习、自然语言处理）以提升完整性。', metadata={}, score=0.9), GraderScore(reason='回答部分相关，指出人工智能（AI）是计算机科学的一个领域，这是正确的。但回答过于简略，缺乏对人工智能具体含义、目标或应用的解释，未能完整解答‘什么是人工智能’这一问题。一个更完整的回答应包括让机器模拟人类智能行为（如学习、推理、识别等）的内容。因此，相关性尚可，准确性基本成立，但完整性不足。', metadata={}, score=0.4), GraderScore(reason='回答准确且相关，简明扼要地解释了人工智能的核心概念——让机器像人一样思考和学习，并列举了关键技术如机器学习、深度学习和自然语言处理，体现了完整性。但可以进一步补充人工智能的应用领域或目标（如推理、识别、决策等），以提升回答的全面性，因此未达到满分。', metadata={}, score

## 4. 结果分析

评估完成后，你可以对结果进行分析。以下是如何处理评估结果的示例：

In [ ]:
# 示例：结果处理代码
def analyze_results(results):
    """分析评估结果的示例函数"""
    print("评估结果分析:")
    for i, sample_result in enumerate(results["results"]):
        print(f"\n样本 {i+1}:")
        for grader_name, scores in sample_result.items():
            print(f"  {grader_name} 评估器:")
            for j, score in enumerate(scores):
                print(f"    回答 {j+1}: 分数={score.score:.2f}, 理由='{score.reason}'")

analyze_results(results)

评估结果分析:

样本 1:
  length_evaluator 评估器:
    回答 1: 分数=0.56, 理由='回答长度为 56 个字符'
    回答 2: 分数=0.14, 理由='回答长度为 14 个字符'
    回答 3: 分数=0.45, 理由='回答长度为 45 个字符'
  keyword_evaluator 评估器:
    回答 1: 分数=0.00, 理由='查询中的 1 个关键词中有 0 个在回答中找到'
    回答 2: 分数=0.00, 理由='查询中的 1 个关键词中有 0 个在回答中找到'
    回答 3: 分数=0.00, 理由='查询中的 1 个关键词中有 0 个在回答中找到'
  quality_evaluator 评估器:
    回答 1: 分数=0.90, 理由='回答准确地定义了人工智能，指出它是计算机科学的一个分支，目标是理解智能本质并制造能模拟人类智能行为的机器。内容相关、表述清晰且信息完整，但可以进一步补充一些典型应用或技术（如机器学习、自然语言处理）以提升完整性。'
    回答 2: 分数=0.40, 理由='回答部分相关，指出人工智能（AI）是计算机科学的一个领域，这是正确的。但回答过于简略，缺乏对人工智能具体含义、目标或应用的解释，未能完整解答‘什么是人工智能’这一问题。一个更完整的回答应包括让机器模拟人类智能行为（如学习、推理、识别等）的内容。因此，相关性尚可，准确性基本成立，但完整性不足。'
    回答 3: 分数=0.85, 理由='回答准确且相关，简明扼要地解释了人工智能的核心概念——让机器像人一样思考和学习，并列举了关键技术如机器学习、深度学习和自然语言处理，体现了完整性。但可以进一步补充人工智能的应用领域或目标（如推理、识别、决策等），以提升回答的全面性，因此未达到满分。'

样本 2:
  length_evaluator 评估器:
    回答 1: 分数=0.64, 理由='回答长度为 64 个字符'
    回答 2: 分数=0.19, 理由='回答长度为 19 个字符'
    回答 3: 分数=0.00, 理由='回答长度为 0 个字符'
  keyword_evaluator 评估器:
    回答 1: 分数=0.00, 理由='查询中的 1 个关键词中有 0 

## 总结

本教程展示了如何使用 RM-Gallery 的 EvaluationRunner 进行模型评估：

1. **创建评估器**：可以创建基于函数的评估器或基于LLM的评估器
2. **准备数据**：使用 DataSample 对象组织测试数据
3. **配置 Runner**：通过 grader_configs 配置多个评估器
4. **运行评估**：使用 runner(data_samples) 异步运行评估任务
5. **分析结果**：处理和分析评估结果

EvaluationRunner 的主要优势：
- 支持并发评估，提高评估效率
- 可以同时运行多个不同的评估器
- 提供统一的接口来处理各种类型的评估任务
- 易于扩展和自定义评估逻辑

在实际使用中，请确保：
- 正确配置API密钥以使用基于LLM的评估器
- 根据需要调整并发数以平衡速度和资源使用
- 根据具体任务选择或创建合适的评估器